In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#import seaborn as sns
from scipy.linalg import sqrtm
from scipy.sparse import csgraph
from scipy import linalg
import itertools

In [2]:
N_TRIAL = 30000
N_ARMS = 100 #N_ARMS -> number of clients
N_FEATURE = 3
M = 29

In [3]:
# X --> TBD now using x_{it} = [1,t,t^2] for all client i
Y_1 = np.genfromtxt('Y(noise0.1).csv',delimiter=',')
W = np.genfromtxt('W(noise0.1).csv',delimiter=',')
Beta = np.genfromtxt('Beta(noise0.1).csv',delimiter=',')

In [4]:
oracle_lst = []
true_choice = []
new_y = -1 * Y_1 + 30 #
for t in np.arange(N_TRIAL):
  # Find indices of M highest arms
  all_reward_t = [new_y.T[t, arm] for arm in np.arange(N_ARMS)]
  chosen_arms = np.array(all_reward_t).argsort()[-M:][::-1]
  # Sum of M highest rewards
  oracle_payoff_t = np.sum([new_y.T[t, choice] for choice in chosen_arms])
  # Append to the list
  oracle_lst.append(oracle_payoff_t)
  true_choice.append(chosen_arms)
oracle_case1 = np.array(oracle_lst)

In [5]:
# Create X_i = [1, t, t^2]
X_1_lst = []
for T in np.arange(N_TRIAL):
  X_1t_lst = []
  for arm in np.arange(N_ARMS):
    temp = []
    temp.append(1)
    temp.append(0.001*(T+1))
    temp.append((0.001*(T+1))**2)
    X_1t_lst.append(np.array(temp))
  X_1_lst.append(np.array(X_1t_lst))
X_1 = np.array(X_1_lst)

In [6]:
def make_regret(payoff, oracle):
    return np.cumsum(oracle - payoff)

def plot_regrets(results, oracle):
    [plt.plot(make_regret(payoff=x['r_payoff'], oracle=oracle), label="alpha: "+str(alpha)) for (alpha, x) in results.items()]

In [7]:
# convert to a sparse matrix -> convert to a long sparse vector with flatten()
def X_to_X_m(X, t, arm_choice, n_arms, n_feature): 
  X_m = np.copy(X[t])
  for arm in np.arange(n_arms): # N x p
    if arm not in arm_choice:
      X_m[arm] = np.zeros(shape=n_feature)
  return X_m

In [8]:
# Create the Adj matrix
def constructAdjMatrix(W, n, threshold): #m
    Adj_mat = np.zeros(shape = (n, n))
    for ui in range(n):
        for uj in range(n):
            Adj_mat[ui][uj] = W[ui][uj]
        # trim the graph
            for i in range(n):
                if W[ui][i] <= threshold:
                    Adj_mat[ui][i] = 0;
#         Adj_mat[ui] /= sum(Adj_mat[ui])
    return Adj_mat

# Create the laplacian matrix
def constructLaplacianMatrix(W, n, Gepsilon):
    G = W.copy()
    #Convert adjacency matrix of weighted graph to adjacency matrix of unweighted graph
    for i in range(n):
        for j in range(n):
            if G[i][j] > 0:
                G[i][j] = 1
    L = csgraph.laplacian(G, normed = False)
    I = np.identity(n = G.shape[0])
    GW = I + Gepsilon*L  # W is a double stochastic matrix
    return GW.T

In [9]:
# Create the F matrix
lda = 0.01 #10
T = 0.5
test_adj = constructAdjMatrix(W, N_ARMS, T) #0.85
test_F = constructLaplacianMatrix(test_adj, N_ARMS, lda)
FInv_Init = sqrtm(np.linalg.inv(np.kron(test_F, np.identity(n=N_FEATURE))))

In [10]:
#upload/download trigger
#UPLOAD
def upload(gammaU, IDclient, A_loc, A_up_buff): #, eta, n_clients, n_feature
    numerator = linalg.det(A_loc[IDclient])
    denominator = linalg.det(A_loc[IDclient] - A_up_buff[IDclient])
    if denominator == 0:
        return False
    else:
        check = numerator/denominator
        return check > gammaU

#DOWNLOAD
def download(gammaD, IDclient, A_gob, A_down_buff): #, eta, n_clients, n_feature
    numerator = linalg.det(A_gob)
    denominator = linalg.det(A_gob - A_down_buff[IDclient]) #+ eta * np.identity(n=n_feature * n_clients)
    if denominator == 0:
        return False
    else:
        check = numerator/denominator
        return check > gammaD

In [11]:
def Fed_GOBLin(alpha, eta, X, Y, m, oracle, gammaU, gammaD, FInv):
    n_trial, n_clients, n_feature = X.shape
    
    # 1.1.output object
    r_payoff = np.empty(n_trial)
    c_payoff = np.empty(n_trial)
    cum_regret = np.empty(n_trial)
    client_choice = np.empty(shape=(n_trial, m), dtype=int)
    p = np.empty(shape=(n_trial, n_clients))
    totalCommCost = 0
    
    # 1.2. local statistics
    A_loc = np.array([eta * np.identity(n=n_feature * n_clients) for _ in np.arange(n_clients)]) #np.zeros((n_feature * n_clients, n_feature * n_clients))
    A_up_buff = np.array([np.zeros((n_feature * n_clients, n_feature * n_clients)) for _ in np.arange(n_clients)]) #np.zeros((n_feature * n_clients, n_feature * n_clients))
    b_loc = np.array([np.zeros(shape=n_feature * n_clients)  for _ in np.arange(n_clients)])
    b_up_buff = np.array([np.zeros(shape=n_feature * n_clients)  for _ in np.arange(n_clients)])
    theta_loc = np.empty(shape=(n_trial, n_clients, n_feature* n_clients))
    
    # 1.3 Global statistics
    A_gob = eta * np.identity(n=n_feature * n_clients) #np.zeros((n_feature * n_clients, n_feature * n_clients))
    A_down_buff = np.array([np.zeros((n_feature * n_clients, n_feature * n_clients)) for _ in np.arange(n_clients)])  #np.zeros((n_feature * n_clients, n_feature * n_clients))
    b_gob = np.zeros(shape=n_feature * n_clients)
    b_down_buff = np.array([np.zeros(shape=n_feature * n_clients)  for _ in np.arange(n_clients)])
    
    # 2. Algorithm
    for t in np.arange(n_trial):
        # Compute estimates prediction (p) for all clients
        for a in np.arange(n_clients):
            inv_A = np.linalg.inv(A_loc[a])
            theta_loc[t, a] = inv_A.dot(b_loc[a])
            X_1_tr = FInv.dot((X_to_X_m(X, t, [a], n_clients, n_feature)).flatten())
            p[t, a] = theta_loc[t, a].dot(X_1_tr) + alpha * np.sqrt(np.dot(np.dot(X_1_tr, inv_A), X_1_tr))
            
        # The central server chooses m best clients
        idx = np.argpartition(p[t], -m)[-m:]
        chosen_clients = idx[np.argsort(-(p[t])[idx])]
        for i in np.arange(m):
          client_choice[t][i] = chosen_clients[i]
        
        # Update local statistics based on following conditions
        for chosen_client in client_choice[t]:
            
            # client local update
            X_1_tr_chosen = FInv.dot((X_to_X_m(X, t, [chosen_client], n_clients, n_feature)).flatten())
            A_loc[chosen_client] = A_loc[chosen_client] + np.outer(X_1_tr_chosen, X_1_tr_chosen)
            b_loc[chosen_client] = b_loc[chosen_client] + Y[t, chosen_client] * X_1_tr_chosen
            A_up_buff[chosen_client] = A_up_buff[chosen_client] + np.outer(X_1_tr_chosen, X_1_tr_chosen)
            b_up_buff[chosen_client] = b_up_buff[chosen_client] + Y[t, chosen_client] * X_1_tr_chosen
            
            # check upload triggering event
            if upload(gammaU, chosen_client, A_loc, A_up_buff): #, eta, n_clients, n_feature
                totalCommCost += 1
                
                # update server's statistics
                A_gob += A_up_buff[chosen_client]
                b_gob += b_up_buff[chosen_client]
                
                # update server's download buffer for other clients
                for clientID in np.arange(n_clients):
                    if clientID != chosen_client:
                        A_down_buff[clientID] += A_up_buff[chosen_client]
                        b_down_buff[clientID] += b_up_buff[chosen_client]
                        
                # clear client's upload buffer
                A_up_buff[chosen_client] = np.zeros((n_feature * n_clients, n_feature * n_clients))
                b_up_buff[chosen_client] = np.zeros(shape=n_feature * n_clients)

                # check download triggering event for all clients
                for cli in np.arange(n_clients):
                    if download(gammaD, cli, A_gob, A_down_buff): #, eta, n_clients, n_feature
                        totalCommCost += 1
                        
                        # update client's local statistics, and clear server's download buffer
                        A_loc[cli] += A_down_buff[cli]
                        b_loc[cli] += b_down_buff[cli]
                        
                        # clear cserver's download buffer
                        A_down_buff[cli] = np.zeros((n_feature * n_clients, n_feature * n_clients))
                        b_down_buff[cli] = np.zeros(shape=n_feature * n_clients)
            
            #else: if do not pass the upload, then the statistics are still the same in local
               
        #else: for other clients not selected at round t, the statistics are still the same in local      
        
        # Cumulative regret
        r_payoff[t] = np.sum([Y[t, choice] for choice in client_choice[t]])      
        cum_regret[t] = np.sum(oracle[0:t+1] - r_payoff[0:t+1])
        if (t+1) % 5000 == 0:
            print('TRIAL:',t,'DONE', '| cum_regret:', cum_regret[t])
            print('Total Communication cost:', totalCommCost)
    
    return dict(A_gob=A_gob, b_gob=b_gob, theta_loc=theta_loc, p=p, client_choice = client_choice, r_payoff=r_payoff, totalCommCost=totalCommCost)

In [12]:
# Run GOBLin
alpha_to_test = [1.25]
results_dict = {alpha: Fed_GOBLin(alpha = alpha, eta = 0.3, X=X_1, Y=(-1 * Y_1 + 30).T, m=M, oracle=oracle_case1, gammaU=1, gammaD=1, FInv=FInv_Init)\
                for alpha in alpha_to_test}

MemoryError: Unable to allocate 6.71 GiB for an array with shape (30000, 100, 300) and data type float64

In [ ]:
# Plot regrets
plt.figure(figsize=(12.5, 7.5))
plot_regrets(results_dict, oracle_case1)
# plt.legend()
# plt.legend(prop={'size': 12}, loc='right')
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.title("Regrets for various levels of alpha")
plt.show()

In [ ]:
# export regret data
# test_lst = [make_regret(payoff=x['r_payoff'], oracle=oracle_case1, t_break=x['t_break']) for (alpha, x) in results_dict.items()]
# df = pd.DataFrame(test_lst[0]) #index 4 is for alpha = 10
# # df.to_csv('C2_GOBLin_regret_66_new.csv', header=False)
# df.to_csv('C2_GOBLin_33_mon.csv', header=False) #(big)

In [ ]:
# 0 = 0, 1 = 1, 2 = 2.5, 3 = 5, 4 = 10, 5 = 20
# export reward
# test_lst = [x['c_payoff'] for (alpha, x) in results_dict.items()]
# df = pd.DataFrame(test_lst[2]) #index 4 is for alpha = 10 
# df.to_csv('C2_GOBLin_reward(big+0.9).csv', header=False)

In [ ]:
## alpha = 1
# TRIAL: 499 DONE | cum_regret: 29704.52123065438
# Total Communication cost: 848
# TRIAL: 999 DONE | cum_regret: 51278.79158387943
# Total Communication cost: 1061
# TRIAL: 1499 DONE | cum_regret: 65316.195153763154
# Total Communication cost: 1061
# TRIAL: 1999 DONE | cum_regret: 65916.0234929869
# Total Communication cost: 1061
# TRIAL: 2499 DONE | cum_regret: 66087.60234283378
# Total Communication cost: 1062
# TRIAL: 2999 DONE | cum_regret: 66203.84549751451
# Total Communication cost: 1062
# TRIAL: 3499 DONE | cum_regret: 66257.25835165591
# Total Communication cost: 1062
# TRIAL: 3999 DONE | cum_regret: 66303.25000551055
# Total Communication cost: 1063
# TRIAL: 4499 DONE | cum_regret: 66392.94680786078
# Total Communication cost: 1063
# TRIAL: 4999 DONE | cum_regret: 66448.51047855684
# Total Communication cost: 1063
# TRIAL: 5499 DONE | cum_regret: 66472.28053063179
# Total Communication cost: 1063
# TRIAL: 5999 DONE | cum_regret: 66498.09887980833
# Total Communication cost: 1063
# TRIAL: 6499 DONE | cum_regret: 66532.42905372415
# Total Communication cost: 1063
# TRIAL: 6999 DONE | cum_regret: 66549.503987295
# Total Communication cost: 1063
# TRIAL: 7499 DONE | cum_regret: 66573.35166771091
# Total Communication cost: 1064
# TRIAL: 7999 DONE | cum_regret: 66610.13307593796
# Total Communication cost: 1064
# TRIAL: 8499 DONE | cum_regret: 66652.87835758101
# Total Communication cost: 1064
# TRIAL: 8999 DONE | cum_regret: 66693.81406239039
# Total Communication cost: 1064
# TRIAL: 9499 DONE | cum_regret: 66733.95679362216
# Total Communication cost: 1064
# TRIAL: 9999 DONE | cum_regret: 66765.02921287144
# Total Communication cost: 1064